In [26]:
import numpy as np
from scipy.io import loadmat
from sklearn.metrics import accuracy_score
import scipy

In [27]:
def softmax_cost(theta, n_classes, input_size, lambda_, data, labels):
    """
    Compute the cost and derivative.
    n_classes: the number of classes
    input_size: the size N of the input vector
    lambda_: weight decay parameter
    data: the N x M input matrix, where each column data[:, i] corresponds to
          a single test set
    labels: an M x 1 matrix containing the labels corresponding for the input data
    """
    # k stands for the number of classes
    # n is the number of features and m is the number of samples
    k = n_classes
    n, m = data.shape
    # Reshape theta
    theta = theta.reshape((k, n))
    # Probability with shape (k, m)
    theta_data = theta.dot(data)
    alpha = np.max(theta_data, axis=0)
    theta_data -= alpha # Avoid numerical problem due to large values of exp(theta_data)
    proba = np.exp(theta_data) / np.sum(np.exp(theta_data), axis=0)
    # Matrix of indicator fuction with shape (k, m)
    labels = np.reshape(labels,(m,))
    indicator = scipy.sparse.csr_matrix((np.ones(m), (labels, np.arange(m))))
    indicator = np.array(indicator.todense())
    # Cost function
    cost = -1.0/m * np.sum(indicator * np.log(proba)) + 0.5*lambda_*np.sum(theta*theta)
    # Gradient matrix with shape (k, n)
    grad = -1.0/m * (indicator - proba).dot(data.T) + lambda_*theta
    # Unroll the gradient matrices into a vector
    grad = grad.ravel()
    return cost, grad

In [28]:
def gradientDescent(J,theta,alpha,options):
    for i in range(options['maxiter']):
        _,grad = J(theta)
        theta = theta - alpha * grad
    return theta

In [29]:
def softmax_train(input_size, n_classes, lambda_, input_data, labels, alpha, options={'maxiter': 400, 'disp': True}):
    """
    Train a softmax model with the given parameters on the given data.
    Returns optimal theta, a vector containing the trained parameters
    for the model.
    input_size: the size of an input vector x^(i)
    n_classes: the number of classes
    lambda_: weight decay parameter
    input_data: an N by M matrix containing the input data, such that
                input_data[:, c] is the c-th input
    labels: M by 1 matrix containing the class labels for the
            corresponding inputs. labels[c] is the class label for
            the c-th input
    options (optional): options
      options['maxiter']: number of iterations to train for
    """
    # initialize parameters
    theta = 0.005 * np.random.randn(n_classes * input_size)
    J = lambda theta : softmax_cost(theta, n_classes, input_size, lambda_, input_data, labels)
    # Find out the optimal theta
    opt_theta = gradientDescent(J,theta,alpha,options=options)
    model = {'opt_theta': opt_theta, 'n_classes': n_classes, 'input_size': input_size}
    return model

In [30]:
def softmax_predict(model, data):
    """
    model: model trained using softmax_train
    data: the N x M input matrix, where each column data[:, i] corresponds to
          a single test set
    pred: the prediction array.
    """
    theta = model['opt_theta'] # Optimal theta
    k = model['n_classes']  # Number of classes
    n = model['input_size'] # Input size (number of features)
    # Reshape theta
    theta = theta.reshape((k, n))
    # Probability with shape (k, m)
    theta_data = theta.dot(data)
    alpha = np.max(theta_data, axis=0)
    theta_data -= alpha # Avoid numerical problem due to large values of exp(theta_data)
    proba = np.exp(theta_data) / np.sum(np.exp(theta_data), axis=0)
    # Prediction values
    pred = np.argmax(proba, axis=0)
    return pred


In [31]:
X = loadmat('./Data/mnistTrainImages.mat')
X = X['trainData'].T
y = loadmat('./Data/mnistTrainLabels.mat')
y = y['trainLabels']
X_test = loadmat('./Data/mnistTestImages.mat')
X_test = X_test['testData'].T
y_test = loadmat('./Data/mnistTestLabels.mat')
y_test = y_test['testLabels']
print X.shape
print X_test.shape

(784, 60000)
(784, 10000)


In [32]:
input_size = 28 * 28 # Size of input vector (MNIST images are 28x28)
n_classes = 10       # Number of classes (MNIST images fall into 10 classes)
lambda_ = 1e-4 # Weight decay parameter

In [25]:
options = {'maxiter': 1000, 'disp': True}
alphas = [0.01,0.1,1,10] #learning rates
for alpha in alphas:
    model = softmax_train(input_size, n_classes, lambda_, X, y, alpha,options)
    pred = softmax_predict(model, X_test)
    print 'for learning rate %f accuracy is %f' % (alpha,accuracy_score(y_test,pred))

for learning rate 0.010000 accuracy is 0.870900
for learning rate 0.100000 accuracy is 0.908400
for learning rate 1.000000 accuracy is 0.922200
for learning rate 10.000000 accuracy is 0.912900


| Number of iterations | Learning rate | Accuracy (%)|
|:--------------------:|:-------------:|:--------:|
|100|0.01|78.75|
|100|0.1|87.06|
|100|1|90.85|
|100|10|90.55|
|500|0.01|84.97|
|500|0.1|89.97|
|500|1|91.94|
|500|10|82.32|
|1000|0.01|87.09|
|1000|0.1|90.84|
|1000|1|92.22|
|1000|10|91.29|

<center> <h3> Some observations </h3> </center>
1. As the number of iterations increase the accuracy increases meaning that the algorithm is learning the actual theta.
2. For learning rate equal to 1, there is maximum accuracy for a given number of iteration.
3. The learning rate is a hyper-parameter and needs to be tuned according to the dataset. For this dataset the optimal value for alpha is 1.